In [32]:
import findspark
findspark.init()
import mysql.connector

from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

import pandas as pd

spark = SparkSession.builder.master("local[*]").appName("Capstone").getOrCreate()

CDW_SAPP_CUSTMER.JSON extracted and transformed 

In [22]:
# Load the JSON file into a PySpark SQL dataframe
credit_cards_df = spark.read.json("json_files\cdw_sapp_custmer.json")

# Add "678" at the beginning of the phone number
new_phone_number = concat(lit("678"), col("CUST_PHONE"))

# Format the phone number into (XXX)XXX-XXXX format
formatted_phone_number = concat(lit("("), substring(new_phone_number, 1, 3), lit(")"),
                        substring(new_phone_number, 4, 3), lit("-"),
                        substring(new_phone_number, 7, 4))

#adds the new number to credit_cards
credit_cards_df = credit_cards_df.withColumn("FORMATTED_PHONE_NUMBER", formatted_phone_number)

# Select the required fields from the dataframe and transform data
CDW_SAPP_CUSTOMER = credit_cards_df.select("SSN",
                                        initcap("FIRST_NAME").alias("FIRST_NAME"),
                                        lower("MIDDLE_NAME").alias("MIDDLE_NAME"),
                                        initcap("LAST_NAME").alias("LAST_NAME"), 
                                        "Credit_card_no",
                                        concat( "APT_NO", lit(", "), "STREET_NAME").alias("ADDRESS"),
                                        "CUST_CITY", "CUST_STATE", "CUST_COUNTRY", "CUST_ZIP",
                                        "CUST_EMAIL","FORMATTED_PHONE_NUMBER", "LAST_UPDATED")

CDW_SAPP_CUSTOMER.show(1)


+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+-------------------+----------------------+--------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  Credit_card_no|             ADDRESS|CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|         CUST_EMAIL|FORMATTED_PHONE_NUMBER|        LAST_UPDATED|
+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+-------------------+----------------------+--------------------+
|123456100|      Alec|         wm|   Hooper|4210653310061055|656, Main Street ...|  Natchez|        MS|United States|   39120|AHooper@example.com|         (678)123-7818|2018-04-21T12:49:...|
+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+-------------------+----------------------+--------------------+
only showing top 1 row



CDW_SAPP_CREDIT.JSON extracted and transformed 

In [20]:
# Load the JSON file into a PySpark SQL dataframe
credit_cards_df2 = spark.read.json("json_files\cdw_sapp_credit.json")

# Convert DAY, MONTH, YEAR to TIMEID in YYYYMMDD format
timeid = concat(col("YEAR"), lpad(col("MONTH"), 2, "0"), lpad(col("DAY"), 2, "0"))
credit_cards_df2 = credit_cards_df2.withColumn("TIMEID", timeid)

# Select the required fields from the dataframe and transform data
CDW_SAPP_CREDIT = credit_cards_df2.select("CREDIT_CARD_NO", "TIMEID", "CUST_SSN", "BRANCH_CODE",
                                          "TRANSACTION_TYPE", "TRANSACTION_VALUE", "TRANSACTION_ID")

CDW_SAPP_CREDIT.show()


+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|  CREDIT_CARD_NO|  TIMEID| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|4210653349028689|20180214|123459988|        114|       Education|             78.9|             1|
|4210653349028689|20180320|123459988|         35|   Entertainment|            14.24|             2|
|4210653349028689|20180708|123459988|        160|         Grocery|             56.7|             3|
|4210653349028689|20180419|123459988|        114|   Entertainment|            59.73|             4|
|4210653349028689|20181010|123459988|         93|             Gas|             3.59|             5|
|4210653349028689|20180528|123459988|        164|       Education|             6.89|             6|
|4210653349028689|20180519|123459988|        119|   Entertainment|            43.39|             7|


CDW_SAPP_BRANCH.JSON extracted and transformed 

In [23]:
# Load the JSON file into a PySpark SQL dataframe
credit_cards_df3 = spark.read.json("json_files\cdw_sapp_branch.json")

# Add "678" at the beginning of the phone number
new_phone_number2 = col("BRANCH_PHONE")

# Format the phone number into (XXX)XXX-XXXX format
formatted_phone_number2 = concat(lit("("), substring(new_phone_number2, 1, 3), lit(")"),
                        substring(new_phone_number2, 4, 3), lit("-"),
                        substring(new_phone_number2, 7, 4))

#adds the new number to credit_cards
credit_cards_df3 = credit_cards_df3.withColumn("BRANCH_PHONE", formatted_phone_number2)

# Add a new column 'BRANCH_ZIP_NEW' based on the value of 'BRANCH_ZIP'
credit_cards_df3 = credit_cards_df3.withColumn(
    "BRANCH_ZIP_NEW",
    when(col("BRANCH_ZIP").isNull(), "00000").otherwise(col("BRANCH_ZIP"))
)

# Select the required fields from the dataframe and transform data
CDW_SAPP_BRANCH = credit_cards_df3.select("BRANCH_CODE","BRANCH_NAME","BRANCH_STREET",
                                          "BRANCH_CITY","BRANCH_STATE","BRANCH_ZIP_NEW",
                                          "BRANCH_PHONE","LAST_UPDATED")

CDW_SAPP_BRANCH.show()

+-----------+------------+-------------------+-----------------+------------+--------------+-------------+--------------------+
|BRANCH_CODE| BRANCH_NAME|      BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP_NEW| BRANCH_PHONE|        LAST_UPDATED|
+-----------+------------+-------------------+-----------------+------------+--------------+-------------+--------------------+
|          1|Example Bank|       Bridle Court|        Lakeville|          MN|         55044|(123)456-5276|2018-04-18T16:51:...|
|          2|Example Bank|  Washington Street|          Huntley|          IL|         60142|(123)461-8993|2018-04-18T16:51:...|
|          3|Example Bank|      Warren Street|SouthRichmondHill|          NY|         11419|(123)498-5926|2018-04-18T16:51:...|
|          4|Example Bank|   Cleveland Street|       Middleburg|          FL|         32068|(123)466-3064|2018-04-18T16:51:...|
|          5|Example Bank|        14th Street|    KingOfPrussia|          PA|         19406|(123)484-970

load CDW_SAPP_CUSTOMER into the creditcard_capstone MariaDB database 

In [ ]:
CDW_SAPP_CUSTOMER.write.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                user="root",\
                                password="password",\
                                url="jdbc:mysql://localhost:3306/creditcard_capstone",\
                                dbtable="creditcard_capstone.CDW_SAPP_CUSTOMER").save()


load CDW_SAPP_BRANCH into the creditcard_capstone MariaDB database 

In [ ]:
CDW_SAPP_BRANCH.write.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                user="root",\
                                password="password",\
                                url="jdbc:mysql://localhost:3306/creditcard_capstone",\
                                dbtable="creditcard_capstone.CDW_SAPP_BRANCH").save()


load CDW_SAPP_CREDIT into the creditcard_capstone MariaDB database 

In [ ]:
CDW_SAPP_CREDIT.write.format("jdbc").options(driver="com.mysql.cj.jdbc.Driver",\
                                user="root",\
                                password="password",\
                                url="jdbc:mysql://localhost:3306/creditcard_capstone",\
                                dbtable="creditcard_capstone.CDW_SAPP_CREDIT").save()


connecting to the DB and creating a cursor

In [33]:
# Establish a connection to the database
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="creditcard_capstone"
)

# Create a cursor object to execute SQL queries
mycursor = mydb.cursor()


Display transactions made by customers living in a given zip code for a given month and year, Order by day in descending order.


In [44]:
def transactions_by_zip_month_year(zip_code, month, year):
    # format the month and year parameters as a string in the format "YYYYMM"
    timeid = year + month
    
    sql = "SELECT CDW_SAPP_CREDIT.CREDIT_CARD_NO, CDW_SAPP_CREDIT.TIMEID, CDW_SAPP_CREDIT.TRANSACTION_TYPE, CDW_SAPP_CREDIT.TRANSACTION_VALUE \
    FROM CDW_SAPP_CREDIT \
    JOIN CDW_SAPP_CUSTOMER ON CDW_SAPP_CREDIT.CUST_SSN = CDW_SAPP_CUSTOMER.SSN \
    WHERE CDW_SAPP_CUSTOMER.CUST_ZIP = %s AND CDW_SAPP_CREDIT.TIMEID LIKE %s \
    ORDER BY CDW_SAPP_CREDIT.TIMEID DESC"
    
    val = (zip_code, f"{timeid}%")
    mycursor.execute(sql, val)
    result = mycursor.fetchall()
    for row in result:
        print(row)


transactions_by_zip_month_year("30236", "09", "2018")



('4210653379463441', '20180928', 'Bills', 49.02)
('4210653312837854', '20180928', 'Grocery', 90.51)
('4210653325199161', '20180928', 'Bills', 43.9)
('4210653316473643', '20180928', 'Gas', 59.94)
('4210653322807170', '20180927', 'Healthcare', 49.42)
('4210653371042040', '20180926', 'Bills', 73.77)
('4210653325199161', '20180926', 'Entertainment', 23.91)
('4210653371042040', '20180925', 'Education', 26.0)
('4210653312837854', '20180924', 'Education', 46.58)
('4210653322807170', '20180923', 'Gas', 26.55)
('4210653312837854', '20180923', 'Healthcare', 49.96)
('4210653325562001', '20180920', 'Entertainment', 56.83)
('4210653325199161', '20180920', 'Entertainment', 74.41)
('4210653312837854', '20180917', 'Education', 82.42)
('4210653316473643', '20180917', 'Healthcare', 61.26)
('4210653371042040', '20180916', 'Entertainment', 85.8)
('4210653379463441', '20180915', 'Entertainment', 77.1)
('4210653312837854', '20180914', 'Healthcare', 5.45)
('4210653312837854', '20180912', 'Grocery', 43.12)
('